In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import datetime
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import torch
from torch import nn

def shifting_ace(df, values_for_shift, future_shift):
    new_df1 = df.iloc[:, 0:9]
    new_columns = {}
    for key, value in values_for_shift.items():
        if key == 'Dst':
            for i in range(0, future_shift+value+1):
                if i < future_shift:
                    new_columns[f'{key} +{future_shift-i}'] = df[key].shift(periods=i, fill_value=np.nan)
                elif i == future_shift:
                    new_columns[key] = df[key].shift(periods=i, fill_value=np.nan)
                else:
                    new_columns[f'{key} -{i - future_shift}'] = df[key].shift(periods=i, fill_value=np.nan)
        else: 
            new_columns[key] = df[key]
            for i in range(1, value+1):
                new_columns[f'{key} -{i}'] = df[key].shift(periods=i, fill_value=np.nan)
    new_df2 = pd.DataFrame(new_columns)
    new_df = pd.concat([new_df1, new_df2], axis=1)
    return new_df


In [13]:
df = pd.read_csv(r"Compare_ACE_DSCOVR.csv", sep=';', na_values='N', decimal=',')

values = ['Dst', 'B_x', 'B_gsm_y', 'B_gsm_z', 'B_magn', 'H_den_SWP', 'SW_spd', 'Trr_SWP']
nums_of_shifts = [24, ]*8

# nums_of_shifts = [2, 3, 2]
future_shifting = 1 #горизонт предсказания
#nums_of_shifts = [43, 26, 12, 3, 19, 56, 16]

# names = list(df.iloc[:, 9:])
# scaler = StandardScaler() #нормировка
# df.iloc[:, 9:] = scaler.fit_transform(df.iloc[:, 9:])
# df.iloc[:, 9:] = pd.DataFrame(df.iloc[:, 9:], columns=names)

values_for_shift = dict(zip(values, nums_of_shifts))
df = shifting_ace(df, values_for_shift, future_shifting) #погружение

# пример как осуществлено погружение
check_df = df.iloc[:10, 7:25]
check_df

,doyCos,hourCos,Dst +1,Dst,Dst -1,Dst -2,Dst -3,Dst -4,Dst -5,Dst -6,Dst -7,Dst -8,Dst -9,Dst -10,Dst -11,Dst -12,Dst -13,Dst -14
0,-0.862961,-0.130526,-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.862598,-0.382683,-15,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.862235,-0.608761,-15,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.861872,-0.793353,-13,-15.0,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.861508,-0.923880,-13,-13.0,-15.0,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.861143,-0.991445,-14,-13.0,-13.0,-15.0,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.860779,-0.991445,-15,-14.0,-13.0,-13.0,-15.0,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.860413,-0.923880,-15,-15.0,-14.0,-13.0,-13.0,-15.0,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.860048,-0.793353,-15,-15.0,-15.0,-14.0,-13.0,-13.0,-15.0,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.859681,-0.608761,-13,-15.0,-15.0,-15.0,-14.0,-13.0,-13.0,-15.0,-15.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#добавляю дату-время на место индексов

new_values = list(df.iloc[:, 9:].columns)

date_values = ['year', 'month', 'day', 'hour from']
date = pd.DataFrame(df.loc[:, date_values])

date['hour from'] = date['hour from'].map(lambda x: '0'+str(x) if x not in range(10, 24) else str(x))
date['datetime'] = date['year'].astype(str) + date['month'].astype(str) + date['day'].astype(str) + ' ' + date['hour from']
date['datetime'] = pd.to_datetime(date['datetime'])

dataset = df[new_values]
dataset = dataset.set_index(date['datetime'])

# dataset_ACE = dataset #отсюда взять значения дст и соединить с данными с discover
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 205356 entries, 1997-10-21 00:00:00 to 2021-03-25 11:00:00
Columns: 201 entries, Dst +1 to Trr_SWP -24
dtypes: float64(200), int64(1)
memory usage: 316.5 MB


In [15]:
dataset = dataset.dropna()

In [6]:
# data_discover = pd.read_csv(r'Discover_с_интреполяцией12_актуальные_данные_до_22.09.2024.csv', sep=';', decimal=',')

# names = ['bx_gsm', 'by_gsm', 'bz_gsm', 'bt', 'proton_density', 'proton_speed', 'proton_temperature']
# nums_of_shifts = [24, ]*7

# Dst - 43, Bx - 26, B_GSM_y - 12, B_GSM_z - 3, B_magn - 19, SW_spd - 56, H_den - 16

# обрезка данных до даты окончания данных с ACE
## 40715 = 25.03.2021 11:00

#data_discover = data_discover.iloc[:40716]
#data_discover

In [ ]:
# values_for_shift_discover = dict(zip(names, nums_of_shifts))
# data_discover = shifting_discover(data_discover, values_for_shift_discover)

# names = list(data_discover)

# scaler = StandardScaler()
# data_discover = scaler.fit_transform(data_discover)
# data_discover_scaled = pd.DataFrame(data_discover, columns=names)
# #data_discover_scaled = data_discover_scaled.set_index(date['datetime'])

# data_discover_scaled
# data_discover = data_dicover_scaled.dropna()

## Значения дст с 27.07.2016 0:00 по 25.03.2021 11:00
# dst_ACE = dataset_ACE.iloc[164496:, 0:44]

# dst_ACE = dataset_ACE.iloc[164496:, 0:1]
# dst_ACE

#data_discover = data_discover + dst_ACE
#print(len(data_discover), len(dst_ACE))

In [16]:
from_index_data = 1

In [17]:
if not dataset.index.is_monotonic_increasing:
   dataset = dataset.sort_index()

#последняя дата - 2021-12-09 23:00:00

# x_train = dataset.loc[:'2018-01-01 01:00:00', dataset.columns[future_shifting:]]
# x_test = dataset.loc['2018-01-01 02:00:00':, dataset.columns[future_shifting:]]
# y_train = dataset['Dst +1'].loc[:'2018-01-01 01:00:00']
# y_test = dataset['Dst +1'].loc['2018-01-01 02:00:00':]

# x_train = dataset.loc[:'2017-10-06 14:00:00', dataset.columns[future_shifting:]]
# x_test = dataset.loc['2017-10-06 14:00:00':, dataset.columns[future_shifting:]]
# y_train = dataset['Dst +1'].loc[:'2017-10-06 14:00:00']
# y_test = dataset['Dst +1'].loc['2017-10-06 14:00:00':]
      
x_train, x_test, y_train, y_test = train_test_split(dataset.iloc[:, from_index_data:], dataset['Dst +1'], test_size=0.15, shuffle=False)
# x_train, x_test, y_train, y_test = x_train.to_numpy(), x_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

In [18]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 172935 entries, 1997-10-22 01:00:00 to 2017-10-15 17:00:00
Columns: 200 entries, Dst to Trr_SWP -24
dtypes: float64(200)
memory usage: 265.2 MB


In [ ]:
plt.plot(y_train, c='blue')
plt.plot(y_test, c='red')
plt.show()

In [ ]:
x_test

In [ ]:
#y_train

In [ ]:
# y_test

In [10]:
pipe = make_pipeline(StandardScaler(), LinearRegression())
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)

# model = linear_model.LinearRegression()
# model.fit(x_train, y_train)

# y_pred = model.predict(x_test)

R_2 = metrics.r2_score(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
MAE = metrics.mean_absolute_error(y_test, y_pred)

print('R**2 score:', R_2)
print('MSE:', MSE)
print('MAE:', MAE)

R**2 score: -0.6448264264272858
MSE: 220.25898296592743
MAE: 2.349411798087127


In [ ]:
len(y_test), len(y_pred)

In [11]:
from sklearn.linear_model import Ridge

lambda_values = [0.00001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 3, 5, 6, 7, 8, 9, 10, 50, 500, 1000, 10000, 50000, 100000, 1000000, 5000000]

for lambda_val in lambda_values:
    ridge_reg = Ridge(lambda_val)
    ridge_reg.fit(x_train, y_train)
    y_pred = ridge_reg.predict(x_test)
    mse_ridge = metrics.mean_squared_error(y_pred, y_test)
    r2_ridge = metrics.r2_score(y_test, y_pred)
    print(('MSE={} and r^2={} with Lambda={}').format(mse_ridge, r2_ridge, lambda_val))

MSE=220.25898295356635 and r^2=-0.6448264263349772 with Lambda=1e-05
MSE=220.25897058518683 and r^2=-0.6448263339717182 with Lambda=0.01
MSE=220.25892106216273 and r^2=-0.6448259641489866 with Lambda=0.05
MSE=220.25885915843025 and r^2=-0.6448255018709279 with Lambda=0.1
MSE=220.25836392996663 and r^2=-0.644821803656884 with Lambda=0.5
MSE=220.25774489801705 and r^2=-0.6448171809164365 with Lambda=1
MSE=220.25712587011805 and r^2=-0.6448125582062374 with Lambda=1.5
MSE=220.2552688106432 and r^2=-0.6447986902565244 with Lambda=3
MSE=220.2527927878817 and r^2=-0.6447802000791172 with Lambda=5
MSE=220.2515548007263 and r^2=-0.644770955171321 with Lambda=6
MSE=220.2503168297357 and r^2=-0.6447617103842389 with Lambda=7
MSE=220.2490788749021 and r^2=-0.6447524657178119 with Lambda=8
MSE=220.24784093622404 and r^2=-0.6447432211720296 with Lambda=9
MSE=220.24660301368982 and r^2=-0.6447339767468048 with Lambda=10
MSE=220.1970993557088 and r^2=-0.6443642986353579 with Lambda=50
MSE=219.6419588

In [ ]:
from catboost import CatBoostRegressor, Pool

model_cat = CatBoostRegressor(iterations=100,
                          depth=2,
                          learning_rate=1,
                          loss_function='RMSE')

train_pool = Pool(x_train, y_train, cat_features=[])
test_pool = Pool(x_test, y_test, cat_features=[])

model_cat.fit(train_pool)
y_pred_cat = model_cat.predict(test_pool)

R_2_catboost = metrics.r2_score(y_test, y_pred_cat)
MSE_catboost = metrics.mean_squared_error(y_test, y_pred_cat)

print('R**2 catboost score:', R_2_catboost)
print('MSE catboost:', MSE_catboost)

In [ ]:
# plt.hist(MSE, color = 'blue', edgecolor = 'black',
#          bins = 5)

# plt.xlabel('ACE')
# plt.ylabel('MSE, нТл')

In [ ]:
# corr_matrix = dataset.corr()

# plt.figure(figsize=(6,5))
# sns.heatmap(corr_matrix)
# plt.show()

In [ ]:
plt.plot(y_test, c='blue')
plt.plot(y_pred, c='red')
plt.show()